* Author: Gordon Erlebacher
* Date: 2022-03-03
* Purpose: convert Sent field to either date (nb seconds from a fixed time) or an empty field.

In [101]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [262]:
# import pandas as pd
import numpy as np
import regex as rex
import pandas as pd
from datetime import datetime
from dateutil import parser
from dateutil.tz import gettz
from unidecode import unidecode
import process_to_library as tolib
import standardize_library as standlib

In [212]:
filenm = "output_0211.csv"
df = pd.read_csv(filenm)

In [213]:
name = "  Nancy Miller "
rex.match(r'\s*(\w+)\s+(\w+)(?=[^@]*$)\W*$', name)

<regex.Match object; span=(0, 15), match='  Nancy Miller '>

In [214]:
name = ' Scof F. (satwell@admin.fsu.edu)'
com = rex.sub(r'\W*(\w+)\s+([A-H])\.', r'\1\2 ', name) 
print("com: ", com)

com = rex.sub(r'\W*\(([^\(]*)\)', r' \1', com)
print("com: ", com)

com = 'Mia Shargel (mshargel@gmail.com)'
com = rex.sub(r'\W*\(([^\(\)]*)\)', r' \1', com)
print("com: ", com)

com = 'Brian T.'
com = rex.sub(r'\b(\w+)\s+([A-Z])\.', r'\1\2 ', com, flags=rex.U) 
print(com)

com = "  Scott Williams  scott@hunterandharp.com "
com = unidecode(com)
com   = rex.match(r'\W*([\w\-]+)\s+([\w\-]+)\s+\W*([\w\.\_]+@[\w\.\-]+)', com, flags=rex.U)
# com = rex.match(r'\W*([\w\-]+)\s+([\w\-]+)', com, flags=rex.U)
print(com)


com:  ScofF  (satwell@admin.fsu.edu)
com:  ScofF satwell@admin.fsu.edu
com:  Mia Shargel mshargel@gmail.com
BrianT 
<regex.Match object; span=(0, 41), match='  Scott Williams  scott@hunterandharp.com'>


In [215]:
name = ' DarrenM  Soto Esq.'


In [216]:
name = " Dr. "
rex.sub(".*\bDr.", '', name)

name = "This island is beautiful"
rex.sub("\bis\b", "gordon", name)


el = 'ads ( gord (adf.;afdlkj com ) asdf'
el = "as (adf (adf ) gas)" # should return "as", but returns "as gas)"
match = rex.match(r'.*\([^@\(\)]*@[^@\)]*\)', el)
if  match:  # there is an email, so remove parenthes
    el = rex.sub(r'\(([^\(\)]*)\)', r'\1', el)
else: 
    # Remove everything within parenthesis along with the parenthesis (knowing that email is excluded)
    el = rex.sub(r'\([^\)]*\)', '', el)
print(el)

as  gas)


In [217]:
df.columns

Index(['filenm', 'From', 'Sent', 'To', 'CC', 'Bcc', 'Subject', 'Attachments',
       'Importance', 'isThread', 'isAutoMessage', 'isDisplacement',
       'hasAllCapLine', 'hasBadDate', 'Body'],
      dtype='object')

In [287]:
cc_list = df['CC'].values   # 170 not processed out of 71,000
to_list = df['To'].values  # 566 not processed out of 71,000

# for nb in range(4):
#     print(f"==> to_list[{nb}]: ", to_list[nb])
    
new_cc_list, cc_not_processed = tolib.process_list(cc_list[0:], debug=False)
new_to_list, to_not_processed = tolib.process_list(to_list[0:], debug=False)
        
print("len: ", len(not_processed), len(new_cc_list), len(cc_list))
print("len: ", len(not_processed), len(new_to_list), len(to_list))
df.columns

71143 71143
71143 71143
len:  0 71143 71143
len:  0 71143 71143


Index(['filenm', 'From', 'Sent', 'To', 'CC', 'Bcc', 'Subject', 'Attachments',
       'Importance', 'isThread', 'isAutoMessage', 'isDisplacement',
       'hasAllCapLine', 'hasBadDate', 'Body'],
      dtype='object')

In [298]:
# Update DataFrame
# Data read in from output_0211.csv
df.columns
df["To"] = new_to_list
df["cc"] = new_cc_list
df.to_csv("output_03112022.csv", index=0)

In [293]:
df = pd.read_csv("output_0211.csv")
df1 = pd.read_csv("output_03112022.csv")

In [297]:
to = df['To'].values
to1 = df1['To'].values
# for n in range(10):
#     print(to[n])
#     print(to1[n])
fromm = df['From'].values
fromm1 = df1['From'].values
for n in range(10):
    print(fromm[n])
    print(fromm[n])

Shane A. Moniz
Shane A. Moniz
Beaudoin, Mark
Beaudoin, Mark
Ken Morris
Ken Morris
Jennifer Naff [mailto:director@springtimetallahassee.com]
Jennifer Naff [mailto:director@springtimetallahassee.com]
Paul LaMaster
Paul LaMaster
Behrman, Allison
Behrman, Allison
Alfano, Michael
Alfano, Michael
Paul LaMaster
Paul LaMaster
McCraw, Rick [mailto:Richard.McCraw@talgov.com]
McCraw, Rick [mailto:Richard.McCraw@talgov.com]
Matt Thompson
Matt Thompson


In [284]:
def reduce_to_email(com):
    if rex.match(r'.*INVALID', com):
        print("... invalid")
        return
    match = standlib.re_email.match(com)
    if match:
        print("==> ", match.groups()[-1])
    else:
        com = rex.sub(" ", "_", com.strip())
        print(com)

In [285]:
# for el in new_cc_list[0:100]:
#     fields = el.split(";")
#     for com in fields:
#         reduce_to_email(com)

In [286]:
for nb in range(5):
    print(f"==> to_list[{nb}]: ", to_list[nb])
    print(f"==> new_to_list[{nb}]: ", new_to_list[nb])

==> to_list[0]:  Brooks Hayes (brooks@culpeppercc.com)
==> new_to_list[0]:  Brooks Hayes  brooks@culpeppercc.com
==> to_list[1]:  Coons, Kristen
==> new_to_list[1]:   Kristen Coons
==> to_list[2]:  kim@inkbridge.com
==> new_to_list[2]:  kim@inkbridge.com
==> to_list[3]:  Faris, Alison
==> new_to_list[3]:   Alison Faris
==> to_list[4]:   andrew.gillum@talgov.com, anita.ﬂowers@talgov.com, rick.fernandez@talgov.com, Nancy Miller, scof, gil.ziﬀer@talgov.com, cur_s.richardson@talgov.com, deslogeb@leoncountyFL.gov, dozierk@leoncountyFl.gov, proctorb@leoncountyFL.gov, dailey@leoncountyFL.gov, Mary Ann Lindley
==> new_to_list[4]:  andrew.gillum@talgov.com; anita.flowers@talgov.com; rick.fernandez@talgov.com; Nancy Miller; gil.ziffer@talgov.com  scof; cur_s.richardson@talgov.com; deslogeb@leoncountyFL.gov; dozierk@leoncountyFl.gov; proctorb@leoncountyFL.gov; dailey@leoncountyFL.gov; Mary Ann Lindley


In [146]:
count = 0
print(len(not_processed))
for el in not_processed:
    if rex.match(r'.*UNKNOWN', el[0]):
    # if True:
        print(f"==> {el}")
        count += 1

0


In [147]:
count, len(new_to_list), len(to_list)

(0, 71346, 71143)

In [15]:
new_to_list[0], '_____', to_list[0]

('Ryan Grindler  rgrindler@101tally.com ; Rosenfeld, David',
 '_____',
 'Ryan Grindler (rgrindler@101tally.com); Rosenfeld, David')

In [16]:
# the number 4 often, but not always, represents 'ti' because of improper translation from pdf I assume. 
# Exception: ...@awards4u.com  . It is obvious that the email is not @awardstiu. 
# I do not handle these exceptions. For the purposes of analysis, it really does not matter if all the emails with 4 are translated
# Another exception: amy4hand@gmail.com, which is correct I believe
# for i, el in enumerate(zip(to_list[0:], new_to_list[0:])):
#     if rex.match('.*([a-zA-Z]4[a-zA-Z])', el[1]):
#         new_to_list[i] = rex.sub(r'([a-zA-Z])4([a-zA-Z])', r'\1ti\2', new_to_list[i])
#         print("==> ", el[0])
#         print("   --> ", new_to_list[i])

In [17]:
for el in zip(to_list[0:1000], new_to_list[0:1000]):
    print("==> ", el[0])
    print("  --> ", el[1])
    
#   -->  Sean Marston  smarston@ucceng.com , Hays Layerd  hlayerd@think3d.net

==>  Ryan Grindler (rgrindler@101tally.com); Rosenfeld, David
  -->  Ryan Grindler  rgrindler@101tally.com ; Rosenfeld, David
==>  Hurley, Patrick
  -->   Patrick Hurley
==>  Jon Brown; Mathieu Cavell; CMR_Team CMR_Team; Victoria Connell; Deborah Craig; Stephanie Holloway; Lindsay Jordan; Angeline Taylor; Jay Townsend; JessicaMiller
  -->  Jon Brown; Mathieu Cavell; CMR_Team CMR_Team; Victoria Connell; Deborah Craig; Stephanie Holloway; Lindsay Jordan; Angeline Taylor; Jay Townsend; JessicaMiller
==>  Paige Carter-Smith; Parade Springtime Tallahassee; Kring, Edward
  -->  Paige Carter-Smith; Parade Springtime Tallahassee; Kring, Edward
==>   alan.williams@myﬂoridahouse.gov, gary@zprgroup.com, village square, William HaZield
  -->  alan.williams@myfloridahouse.gov; gary@zprgroup.com; village square; William HaZield
==>  nan
  -->  INVALID_TYPE
==>  Daniels, Dustin; Simpson, Roxanne M
  -->  Daniels, Dustin; Simpson, Roxanne M
==>   DeslogeB@leoncountyﬂ.gov, andrew.gillum@talgov.com, ric

In [18]:
name = '  @ asf'
rex.match(r'[^\w]+@', name)

<regex.Match object; span=(0, 3), match='  @'>

In [19]:
name = '2/16/17 3:25 PM (GMT-05:00)'
name = rex.sub(r'(\d?\/\d)\/(\d\d )', r'\1/20\2', name)
rex.match(r'(\d+?\/\d+?)\/(\d\d )', name)

<regex.Match object; span=(0, 8), match='2/16/17 '>

In [ ]:
exceptions

* 'Friday, May 13, 201611:18 AM': datetime.datetime(2022, 5, 13, 18, 16, 11),
* 